### Yonatan Klausner, Jonathan Rawson, and Yair Fax 
## Final Project 

In [28]:
# Imports needed for project 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import pandas as pd
import numpy as np
import pylab as pl
%matplotlib inline
from IPython.core.pylabtools import figsize
import matplotlib.pyplot as plt
figsize(14, 7)

# np.corrcoef(teamTable['HustleScore'], teamTable['Wins'])[0,1] 


# hustle stat - average mph, miles run, balls deflected, contested shots, stats per minute (steals, o rebounds, feet), 
# jump balls, Loose balls recovered, boxouts, screen assists, charges, team, average salary, all nba or all star teams 
# https://stats.nba.com/players/speed-distance/?sort=AVG_SPEED&dir=1

# To do:
# 1) change the way calculate hustle score 
#        a) per minute 
#        b) add in distance and speed (https://stats.nba.com/players/speed-distance/?sort=DIST_MILES_DEF&dir=1&Season=2017-18&SeasonType=Regular%20Season)
#        c) standardize or not needed?
# 2) change up scatter plots 
# 3) multiple webdrivers at once? possible? good idea?
# 4) sort by position
# 5) trends over time - playoff teams, non-playoff teams, championship teams
# 6) predict current season outcomes
# 7) more detailed comparisons with wins
# 8) look at specific player hustle in a team as correlated with wins
# 9) ML 

# GP >= 40 make sure not for current season


## Part 1: Get Data
In this section we get all the data that we will be using for our project.  
In the end, we will have four tables which we will explain in depth later on.

### Step 1: Define Functions for Getting Different Data

In [29]:
def getHustleStats(year):
    return getYearStats("https://stats.nba.com/players/hustle/?sort=MIN&dir=-1&Season=" + year + "&SeasonType=Regular%20Season", year)

def getRegularStats(year):
    return getYearStats("https://stats.nba.com/leaders/?Season=" + year + "&SeasonType=Regular%20Season", year)

def getYearStats(url, year):
    # Use an automated browser so that the webpage is rendered properly
    browser = webdriver.Safari()
    browser.get(url)
    
    # Edit HTML so that we can get the whole table
    browser.execute_script('document.getElementsByClassName("stats-table-pagination__select")[0].setAttribute("id", "btn")')
    browser.execute_script('document.getElementById("btn").children[0].setAttribute("id", "select-all")')
    nextButton = browser.find_element_by_id('btn')
    allButton = browser.find_element_by_id('select-all')
    
    # Click on buttons to get whole table
    nextButton.click()
    allButton.click()
    
    # Get HTML and parse
    innerHTML = browser.execute_script("return document.body.innerHTML")
    root = BeautifulSoup(innerHTML, "lxml")
    table = pd.read_html(str(root.find("table")))
    table = table[0]
    
    # Add year column for later table merging
    table['year'] = year
    browser.close()
    return table

def getPositions(year):
    url = "https://www.basketball-reference.com/leagues/NBA_" + str(year) + "_per_game.html"
    r = requests.get(url)
    root = BeautifulSoup(r.content, "lxml")
    data = root.find("table")
    positions_table = pd.read_html(str(data))[0]
    positions_table['year'] = str(year - 1) + "-" + str(year - 2000)
    return positions_table

def getTeams(filename, year):
    teamTable = pd.read_csv(filename)
    teamTable['year'] = year
    return teamTable

### Step 2: Get Hustle Data on Players
This data consists of several statistics that are related to player hustling.  The statistics we will use from this data are:  

1. **Screen assists** - a screen is a blocking move by an offensive player, by standing beside or behind a defender, to free a teammate to shoot, receive a pass, or drive in to score. A screen assist is a screen that directly leads to a made field goal.  

2. **Deflections** - a deflection occurs when a defensive player redirects the intended direction of the ball.

3. **Loose Balls Recovered** - a loose ball is when neither team is in control of the ball.  A loose ball recovered is when a player recovers a loose ball.   

4. **Charges Drawn** - a charge, or player-control foul, occurs when a dribbler charges into a defender who has already established his position.  A drawn charge is when a defender takes a charge on an offensive player, meaning the defensive player is in position and the offensive player charges into him.  

5. **Contested Shots** - a contested shot is when a defender is within 4ft of the person shooting.   

In [30]:
hustle16_17 = getHustleStats("2016-17")
hustle17_18 = getHustleStats("2017-18")

In [19]:
hustleStats = hustle16_17.append(hustle17_18)
hustleStats['Player'] = stripChars(hustleStats['Player'])
hustleStats.head()

,Player,TEAM,AGE,GP,MIN,ScreenAssists,ScreenAssists PTS,Deflections,OFF Loose BallsRecovered,DEF Loose BallsRecovered,Loose BallsRecovered,% Loose BallsRecovered OFF,% Loose BallsRecovered DEF,ChargesDrawn,Contested2PT Shots,Contested3PT Shots,ContestedShots,year
0,Danuel House,WAS,24,1,0.8,0.0,0.0,0.0,0,0,0.0,0,0,0,0.0,0.0,0.0,2016-17
1,John Lucas,MIN,34,5,2.1,0.0,0.0,0.8,0,0,0.2,0,0,0,0.0,0.0,0.0,2016-17
2,Larry Sanders,CLE,28,5,2.5,0.2,0.4,0.2,0,0,0.2,0,0,0,1.4,0.0,1.4,2016-17
3,Steve Novak,MIL,34,8,2.7,0.0,0.0,0.0,0,0,0.0,0,0,0,0.4,0.1,0.5,2016-17
4,Brice Johnson,LAC,23,3,3.1,0.3,1.0,0.7,0,0,0.0,0,0,0,1.3,0.3,1.7,2016-17


### Step 3: Get Data for Regular Statistics on Players 
This data consists of several common, player statistics such as **MIN** (minutes per game), **PTS** (points per game), **FGM** (field goals made per game), **REB** (rebounds per game), etc. 

In [20]:
regular16_17 = getRegularStats("2016-17")
regular17_18 = getRegularStats("2017-18")
regularStats = regular16_17.append(regular17_18)
regularStats.head()

,#,Player,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF,year
0,1,Russell Westbrook,81,34.6,31.6,10.2,24.0,42.5,2.5,7.2,...,84.5,1.7,9.0,10.7,10.4,1.6,0.4,5.4,33.8,2016-17
1,2,James Harden,81,36.4,29.1,8.3,18.9,44.0,3.2,9.3,...,84.7,1.2,7.0,8.1,11.2,1.5,0.5,5.7,32.4,2016-17
2,3,Isaiah Thomas,76,33.8,28.9,9.0,19.4,46.3,3.2,8.5,...,90.9,0.6,2.1,2.7,5.9,0.9,0.2,2.8,24.7,2016-17
3,4,Anthony Davis,75,36.1,28.0,10.3,20.3,50.5,0.5,1.8,...,80.2,2.3,9.5,11.8,2.1,1.3,2.2,2.4,31.1,2016-17
4,5,DeMar DeRozan,74,35.4,27.3,9.7,20.9,46.7,0.4,1.7,...,84.2,0.9,4.3,5.2,3.9,1.1,0.2,2.4,22.7,2016-17


### Step 4: Get Data for Positions of Players
This data consists of the positions for each player.  The five positions are:
1. PG - point guard
2. SG - shooting guard
3. SF - small forward 
4. PF - power forward
5. C - center

Typically, the **point guard** is the leader of the team when on the court. This position requires substantial ball handling skills and the ability to facilitate the team during a play. The **shooting guard**, as the name implies, is often the best shooter and is probably capable of shooting accurately from longer distances. Generally, they also have good ball-handling skills. The **small forward** often has an aggressive approach to the basket when handling the ball. The **power forward** and the center are usually called "low post" players to the basket, often acting as their team's primary rebounders or shot blockers, or receiving passes to take inside shots. The **center** is typically the larger of the two.

In [39]:
positions16_17 = getPositions(2017)[['Player', 'Pos', 'year']]
positions17_18 = getPositions(2018)[['Player', 'Pos', 'year']]

In [40]:
positionsTable = positions16_17.append(positions17_18)
positionsTable.head()

,Player,Pos,year
0,Alex Abrines,SG,2016-17
1,Quincy Acy,PF,2016-17
2,Quincy Acy,PF,2016-17
3,Quincy Acy,PF,2016-17
4,Steven Adams,C,2016-17


### Step 5: Get Data for Team Wins 

This data consists of team statistics such as wins, losses, home record, road record, etc.  
We use this data to get the number of wins for a team in a given year

In [23]:
teamsWins16_17 = getTeams('teams_2016_2017.csv', '2016-17') # http://media.nba.com/Stats/Standings.aspx?&leagueid=00&seasonid=22016
teamsWins17_18 = getTeams('teams_2017_2018.csv', '2017-18') # http://media.nba.com/Stats/Standings.aspx?&leagueid=00&seasonid=22017
teamsWins16_17.head()

,Team,W,L,Win%,GB,Conf,Div,Home,Road,OT,Last 10,Streak,year
0,BOS,53,29,0.646,0,36-16,11-5,30-11,23-18,2-1,7-3,W 3,2016-17
1,CLE,51,31,0.622,2,35-17,8-8,31-10,20-21,3-4,4-6,L 4,2016-17
2,TOR,51,31,0.622,2,34-18,14-2,28-13,23-18,3-1,8-2,W 4,2016-17
3,WAS,49,33,0.598,4,32-20,8-8,30-11,19-22,4-3,5-5,L 1,2016-17
4,ATL,43,39,0.524,10,30-22,6-10,23-18,20-21,5-0,6-4,L 1,2016-17


## Part 2: Clean Data
After getting all the data needed for our project, we need to clean the data so that we are able to use each table easily.   
First, we merge the hustleStats table and the positionsTable on the player and year columns in order to add the positions to the hustleStats table.  
We then print out any players in the merged table with a null position. 

In [24]:
hustleStatsTemp = pd.merge(hustleStats, positionsTable, on=['Player', 'year'], how='left')
hustleStatsTemp[pd.isnull(hustleStatsTemp['Pos'])]

,Player,TEAM,AGE,GP,MIN,ScreenAssists,ScreenAssists PTS,Deflections,OFF Loose BallsRecovered,DEF Loose BallsRecovered,Loose BallsRecovered,% Loose BallsRecovered OFF,% Loose BallsRecovered DEF,ChargesDrawn,Contested2PT Shots,Contested3PT Shots,ContestedShots,year,Pos
158,Juancho Hernangomez,DEN,21,62,13.6,0.3,0.7,0.7,0.0,0.0,0.5,0.0,0.0,0.00,3.1,1.7,4.7,2016-17,NaN
244,Taurean Prince,ATL,23,59,16.6,0.1,0.2,1.4,0.0,0.0,0.6,0.0,0.0,0.05,2.9,2.4,5.2,2016-17,NaN
275,Nene,HOU,34,67,17.9,2.8,6.5,1.5,0.0,0.0,0.7,0.0,0.0,0.15,4.9,1.7,6.6,2016-17,NaN
601,Vincent Hunter,MEM,23,4,1.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.5,0.5,1.0,2017-18,NaN
669,Walter Lemon,NOP,25,5,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.4,0.2,1.6,2017-18,NaN
745,Juancho Hernangomez,DEN,22,25,11.1,0.2,0.4,0.5,0.2,0.2,0.3,50.0,50.0,0.04,2.3,1.5,3.8,2017-18,NaN
840,Nene,HOU,35,52,14.6,2.0,4.9,1.4,0.1,0.3,0.5,25.0,75.0,0.04,3.5,1.9,5.4,2017-18,NaN
882,Wes Iwundu,ORL,23,62,16.5,0.1,0.2,0.8,0.1,0.3,0.5,30.0,70.0,0.05,1.8,1.7,3.5,2017-18,NaN
1162,Taurean Prince,ATL,24,82,30.0,0.2,0.5,1.8,0.5,0.6,1.1,43.3,56.7,0.11,3.8,2.8,6.6,2017-18,NaN


As seen above, there are a few players whose position was not set properly.  
After looking closely into each case, we realized what was going wrong for each of them.
1. **Juancho Hernangomez** -  
2. **Taurean Prince** - in one table he was named "Taurean Prince" and in the other table it had "Taurean Waller-Prince".
3. **Nene** - in one table he was named "Nene" and in the other table it had "Nene Hilario".  
4. **Vincent Hunter** - 
5. **Walter Lemon** - 
6. **Wes Iwundu** - 

First, we dropped all duplicated rows with the same player and year.   
Then we stripped certain characters in player names such as "Jr.", "I", and "II" because one table had "Jr.", "I", and "II" and the other did not.  Therefore, when merging the tables on the player names, the merge took those players has two different players as opposed to the same player.  
Next, we took care of Nene and Taurean Prince cases by renaming them in the table. 

In [43]:
def stripChars(series):
    return series.str.replace('.', '').str.replace('Jr', '').str.replace(',', '').str.replace('III', '').str.replace('II', '').str.replace('IV', '').str.rstrip()

In [42]:
positionsTable = positionsTable.drop_duplicates(subset=['Player', 'year'])
positionsTable['Player'] = stripChars(positionsTable['Player'])
positionsTable.loc[positionsTable['Player'] == 'Nene Hilario', 'Player'] = 'Nene'
positionsTable.loc[positionsTable['Player'] == 'Taurean Waller-Prince', 'Player'] = 'Taurean Prince'
positionsTable.head()

,Player,Pos,year
0,Alex Abrines,SG,2016-17
1,Quincy Acy,PF,2016-17
4,Steven Adams,C,2016-17
5,Arron Afflalo,SG,2016-17
6,Alexis Ajinca,C,2016-17


After cleaning the data some more, we then merge the hustleStats table and the postionsTable again.  
We then print the rows in the hustleStats that have a null position. 

In [45]:
hustleStats = pd.merge(hustleStats, positionsTable, on=['Player', 'year'], how='left')
hustleStats[pd.isnull(hustleStats['Pos'])]

NameError: name 'positionsTable' is not defined

In [26]:
hustleStats = hustleStats.dropna(subset=['Pos'])
hustleStats.head()

,Player,TEAM,AGE,GP,MIN,ScreenAssists,ScreenAssists PTS,Deflections,OFF Loose BallsRecovered,DEF Loose BallsRecovered,Loose BallsRecovered,% Loose BallsRecovered OFF,% Loose BallsRecovered DEF,ChargesDrawn,Contested2PT Shots,Contested3PT Shots,ContestedShots,year,Pos
0,Danuel House,WAS,24,1,0.8,0.0,0.0,0.0,0,0,0.0,0,0,0,0.0,0.0,0.0,2016-17,SG
1,John Lucas,MIN,34,5,2.1,0.0,0.0,0.8,0,0,0.2,0,0,0,0.0,0.0,0.0,2016-17,PG
2,Larry Sanders,CLE,28,5,2.5,0.2,0.4,0.2,0,0,0.2,0,0,0,1.4,0.0,1.4,2016-17,C
3,Steve Novak,MIL,34,8,2.7,0.0,0.0,0.0,0,0,0.0,0,0,0,0.4,0.1,0.5,2016-17,PF
4,Brice Johnson,LAC,23,3,3.1,0.3,1.0,0.7,0,0,0.0,0,0,0,1.3,0.3,1.7,2016-17,PF


Next, we clean the hustleStats table. 
1. Drop any players that average less than 15 minutes per game. 
2. Drop any players that played less than 40 games in the season. 
3. Change the "TEAM" column to be named 'Team' 
4. Change any player with the two positions of SF and SG to just have a position of SF 
5. Drop unused columns 
6. Drop duplicated players within the same year 

In [33]:
hustleStats = hustleStats[hustleStats['MIN'] >= 15] # Delete players who did not average 15 minutes per game 
hustleStats = hustleStats[hustleStats['GP'] >= 40] # Delete players who did not play 40 games 
hustleStats = hustleStats.rename({'TEAM': 'Team'}, axis=1) # Rename column 
hustleStats.loc[hustleStats['Pos'] == 'SF-SG', 'Pos'] = 'SF' # Change position of player from multiple positions to one 
# Drop unused columns 
hustleStats = hustleStats.drop(['ScreenAssists PTS', 'OFF Loose BallsRecovered', 'DEF Loose BallsRecovered', '% Loose BallsRecovered OFF', '% Loose BallsRecovered DEF'], axis=1)
hustleStats = hustleStats.drop_duplicates(subset=['Player', 'year'])
hustleStats.head()

ValueError: labels ['ScreenAssists PTS' 'OFF Loose BallsRecovered' 'DEF Loose BallsRecovered'
 '% Loose BallsRecovered OFF' '% Loose BallsRecovered DEF'] not contained in axis

In [34]:
teamWinsTable = teamsWins16_17.append(teamsWins17_18)
# Drop unused columns 
teamWinsTable = teamWinsTable.drop(['L', 'Win%', 'GB', 'Conf', 'Div', 'Home', 'Road', 'OT', 'Last 10', 'Streak'], axis=1)
teamWinsTable = teamWinsTable.rename({'W': 'Wins'}, axis=1) # Rename column
teamWinsTable.head()

TypeError: rename() got an unexpected keyword argument "axis"

Now, we have 4 clean tables:   
1. **hustleStats** - players with different hustle stats
2. **regularStats** - players with normal statistics
3. **positionsTable** - players with their positions 
4. **teamsWinsTable** - teams with their wins

## Part 3: Calculate Hustle Score

In order to calculate our hustle score, we will use the sum of screen assists, deflections, loose balls recovered, charges drawn, and contested shots.  However, if we just took the sum of those statistics for each player, certain statistics, such as contested shots, would effect the overall hustle score more than others because its numbers are generally higher.  Therefore, we decided to normalize each statistic by dividing its value by the max for that specific statistic.  

In [ ]:
maxScreenAssists = (hustleStats['ScreenAssists']/hustleStats['MIN']).max()
maxDeflections = (hustleStats['Deflections']/hustleStats['MIN']).max()
maxLooseBallsRecovered = (hustleStats['Loose BallsRecovered']/hustleStats['MIN']).max()
maxChargesDrawn = (hustleStats['ChargesDrawn']/hustleStats['MIN']).max()
maxContestedShots = (hustleStats['ContestedShots']/hustleStats['MIN']).max()
print("maxScreenAssists per minute: " + str(maxScreenAssists))
print("maxDeflections per minute: " + str(maxDeflections))
print("maxLooseBallsRecovered per minute: " + str(maxLooseBallsRecovered))
print("maxChargesDrawn per minute: " + str(maxChargesDrawn))
print("maxContestedShots per minute: " + str(maxContestedShots))

In [ ]:
hustleStats['HustleScore'] = (hustleStats['ScreenAssists']/maxScreenAssists + hustleStats['Deflections']/maxDeflections + hustleStats['Loose BallsRecovered']/maxLooseBallsRecovered + hustleStats['ChargesDrawn']/maxChargesDrawn + hustleStats['ContestedShots']/maxContestedShots)/(hustleStats['MIN'])
hustleStats.head()

In [ ]:
# Sort table by HustleScore
hustleStats = hustleStats.sort_values(by=['HustleScore'],ascending=False)
hustleStats.head(5)

## Part 4: Look for Potential Correlation Between HustleScore and Other Parameters

In [ ]:
scatter = hustleStats.plot.scatter('AGE', 'HustleScore')

(m, b) = np.polyfit(hustleStats['AGE'], hustleStats['HustleScore'], 1)
# Calculate the actual best fit line 
regression_line = [(m*x)+b for x in hustleStats['AGE']]
scatter.plot(hustleStats['AGE'], regression_line, color='red') 

# Label the plot
scatter.set_title("Distribution of Hustle Scores Across Age (2016-2018) with (m, b) = (" + str(round(m,2)) + ", " + str(round(b, 2)) + ")")
scatter.set_xlabel("Age")
scatter.set_ylabel("Hustle Scores")

In [ ]:
scatter = hustleStats.plot.scatter('MIN', 'HustleScore')

(m, b) = np.polyfit(hustleStats['MIN'], hustleStats['HustleScore'], 1)
# Calculate the actual best fit line 
regression_line = [(m*x)+b for x in hustleStats['MIN']]
scatter.plot(hustleStats['MIN'], regression_line, color='red') 

# Label the plot
scatter.set_title("Distribution of Hustle Scores Across Minutes Per Game (2016-2018) with (m, b) = (" + str(round(m,2)) + ", " + str(round(b, 2)) + ")")
scatter.set_xlabel("Minutes Per Game")
scatter.set_ylabel("Hustle Scores")

In [ ]:
teamHustleTable = pd.DataFrame(hustleStats.groupby(['Team', 'year'])['HustleScore'].sum())
teamHustleTable = teamHustleTable.sort_values(by=['HustleScore'],ascending=False)
teamHustleTable = teamHustleTable.reset_index()
teamHustleTable.head()

In [ ]:
teamTable = pd.merge(teamHustleTable, teamWinsTable, on=['Team', 'year'])
teamTable.head()

In [ ]:
# Convert type of wins column to int32 because cannot be of object type when plotting 
teamTable['Wins'] = teamTable['Wins'].astype('int32')
x = teamTable['HustleScore']
y = teamTable['Wins']
fig, ax = plt.subplots()
ax.scatter(x, y) 

# Get the slope and intercept for the best fit line 
(m, b) = np.polyfit(teamTable['HustleScore'], teamTable['Wins'], 1)
# Calculate the actual best fit line 
regression_line = [(m*x)+b for x in teamTable['HustleScore']]
ax.plot(teamTable['HustleScore'], regression_line, color='red') 

# Label the plot
ax.set_title("Distribution of Hustle Scores Across Team Wins (2016-2018) with (m, b) = (" + str(round(m,2)) + ", " + str(round(b, 2)) + ")")
ax.set_xlabel("Team Hustle Score")
ax.set_ylabel("Team Wins")

In [ ]:
x = hustleStats['Pos']
y = hustleStats['HustleScore']
fig, ax = plt.subplots()
ax.scatter(x, y)

In [ ]:
topTeams = teamTable[teamTable['Wins'] >= teamTable['Wins'].median()]
topTeams = topTeams[topTeams['HustleScore'] >= topTeams['HustleScore'].median()]
topTeams.head()

In [ ]:
bottomTeams = teamTable[teamTable['Wins'] < teamTable['Wins'].median()]
bottomTeams = bottomTeams[bottomTeams['HustleScore'] < bottomTeams['HustleScore'].median()]
bottomTeams.head()

In [ ]:
topTeamsPlayers = pd.merge(topTeams, hustleStats, how='left', on=['Team']).sort_values('Pos')
x = topTeamsPlayers['Pos']
y = topTeamsPlayers['HustleScore_y']
fig, ax = plt.subplots()
ax.set_yticks(range(1,3,1))
ax.scatter(x, y)

In [ ]:
bottomTeamsPlayers = pd.merge(bottomTeams, hustleStats, how='left', on=['Team']).sort_values('Pos')
x = bottomTeamsPlayers['Pos']
y = bottomTeamsPlayers['HustleScore_y']
fig, ax = plt.subplots()
ax.set_yticks(range(1,3,1))
ax.scatter(x, y) 

### Part 5: Sources  
1. NBA.com - used for getting hustle statistics, regular statistics, and team statistics
2. **INSERT_SITE** - used for getting player positions 
3. Wikipedia - used for defining certain basketball terms 